In [ ]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.model_selection import cross_val_score

import sys
sys.path.append("../")

from scripts.pipeline_modeling import (
    run_train_pipeline, 
    get_pipeline,
    dvc_get_data)
from scripts.clean_data import CleanDataFrame
from sklearn.utils import shuffle

import os
import mlflow

In [ ]:
df = dvc_get_data(path="Data/ABtwoCampaignEngView.csv", version="v6")

In [ ]:
df.info()


In [ ]:
cleaner = CleanDataFrame()
cleaner.percent_missing(df)

In [ ]:
display(df[df.duplicated('auction_id', False)].head())
print("Duplicated rows:", df[df.duplicated('auction_id', False)].shape[0])

In [ ]:
clean_df = cleaner.drop_duplicates(df)
clean_df = cleaner.drop_unresponsive(clean_df)
clean_df = cleaner.drop_columns(df=clean_df,
                columns=['Unnamed: 0', 'auction_id', 'date', 'no'])
clean_df.info()

In [ ]:
unique_device_make = clean_df.device_make.unique().shape
unique_browsers = clean_df.browser.unique().shape
unique_os = clean_df.platform_os.unique().shape

print(f"unique_device_make: {unique_device_make[0]}")
print(f"unique_browsers: {unique_browsers[0]}")
print(f"unique_os: {unique_os[0]}")